# From start to finish
Generate a movie of the projections and reconstructions from a selected sample

In [ ]:
import matplotlib.pylab as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import glob
import os
import dask
import dask_image.imread
import math
import skimage
import numpy
# import skimage.transform
# from tqdm import tqdm_notebook
# import time
from IPython import display

In [ ]:
# Load our own log file parsing code
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# Display and output defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation

In [ ]:
# Setup scale bar
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = '#E6002E'  # unibe red as default scalebar color

In [ ]:
seaborn.set_context('talk')

The original scans are in the archive at `DKF Lung-Metastasis\Overview\
Get the files to the FastSSD with 
````
rsync --verbose --recursive --compress --times --update --omit-dir-times --prune-empty-dirs --include="*/" --include="*.?og" --include="*.c?v" --include="*.?if" --include="*.png" --exclude="*" ~/research-storage-uct/Archiv_Tape/DKF\ Lung-Metastasis/Overview/KP-TNIKWT02/ /media/habi/Fast_SSD/DKF\ Lung-Metastasis/Overview/KP-TNIKWT02/
````
If the PNGs in the `rec` folder are no longer there, you'll need to reconstruct them again :)

In [ ]:
fastSSD = True
if fastSSD:
    Root = '/media/habi/Fast_SSD/'
else:
    Root = '/home/habi/1272/'
BaseFolder = os.path.join(Root, 'DKF Lung-Metastasis', 'Overview')
Sample = 'KP-TNIKWT02'

In [ ]:
print(BaseFolder)

In [ ]:
os.path.join(BaseFolder, Sample, 'proj', '*.log')

In [ ]:
os.path.join(BaseFolder, Sample, 'proj', Sample + '0*.tif')

In [ ]:
# Get file names
Logfile = sorted(glob.glob(os.path.join(BaseFolder, Sample, 'proj', '*.log')))[0]
Projections = dask_image.imread.imread(os.path.join(BaseFolder, Sample, 'proj', '*[0.123456789].tif'))  # Exclude `KP-TNIKWT2_arc.tif`
Reconstructions = dask_image.imread.imread(os.path.join(BaseFolder, Sample, 'rec', '*.png'))

In [ ]:
# Dicard some of the lower reconstructions, which only contain the sample holder and light up if we present them with equalized histogram
Reconstructions = Reconstructions[215:]

In [ ]:
Projections

In [ ]:
Reconstructions

In [ ]:
if not Logfile:
    print('Please mount the fast SSD...')

In [ ]:
Pixelsize = pixelsize(Logfile)
print('We scanned the sample with a voxel size of %0.2f um' % Pixelsize)

In [ ]:
# Details of the output movie
Seconds = 15
NumberOfFrames = 24 * Seconds # show animation for $Seconds seconds with a frame rate of 24 fps
WidthOfOutput = 800

In [ ]:
print('We have %s projections' % (len(Projections)))
print('and %s reconstructions' % (len(Reconstructions)))

In [ ]:
# Select exactly this many evenly spaced elements from array
# https://stackoverflow.com/a/50685454/323100
# idx = numpy.round(numpy.linspace(0, len(Projections) - 1, NumberOfFrames)).astype(int)

In [ ]:
print('Of these projections and reconstructions I will work with approximately %s equally spaced images' % NumberOfFrames)

In [ ]:
# Show middle images
plt.subplot(121)
plt.imshow(Projections[Projections.shape[0] // 2])
plt.gca().add_artist(ScaleBar(Pixelsize, 'um'))
plt.title('Middle Projection')
plt.axis('off')
plt.subplot(122)
plt.imshow(Reconstructions[Reconstructions.shape[0] // 2])
plt.gca().add_artist(ScaleBar(Pixelsize, 'um'))
plt.title('Middle Reconstruction')
plt.axis('off')
plt.show()

In [ ]:
# Make output directories
os.makedirs(os.path.join(BaseFolder, Sample, 'mov', 'mov_proj'), exist_ok=True)
os.makedirs(os.path.join(BaseFolder, Sample, 'mov', 'mov_rec'), exist_ok=True)

Write out single frames *with* a scale bar of fixed length (5 mm for example).

In [ ]:
# Delete files from previous runs
# Based on https://stackoverflow.com/a/37994379/323100
for file in os.scandir(os.path.join(BaseFolder,
                                    Sample,
                                    'mov',
                                    'mov_proj')):
    os.remove(file.path)
for file in os.scandir(os.path.join(BaseFolder,
                                    Sample,
                                    'mov',
                                    'mov_rec')):
    os.remove(file.path)

In [ ]:
# Let's write the output image with the chosen width (and scaled height)
# Scale the output figure size exactly: https://stackoverflow.com/a/13714720/323100
# Write canvas containts to image: https://stackoverflow.com/a/14913405/323100
for c, projection in enumerate(Projections[numpy.round(numpy.linspace(0, len(Projections) - 1, NumberOfFrames)).astype(int)]):
    # Generate a figure with the chosen width and accordingly scaled height
    fig = plt.figure(figsize=(WidthOfOutput / plt.rcParams['figure.dpi'],
                              round(WidthOfOutput * (projection.shape[0] / projection.shape[1])) / plt.rcParams['figure.dpi']),
                     dpi=plt.rcParams['figure.dpi'])
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    # Equalize histogram for display reasons
    ax.imshow(skimage.exposure.equalize_adapthist(projection))
    ax.add_artist(ScaleBar(Pixelsize, 'um', fixed_value=5, fixed_units='mm'))
    s = '_'
    fig.canvas.print_jpg(os.path.join(BaseFolder, Sample, 'mov', 'mov_proj',
                                      s.join([Sample,
                                              str(NumberOfFrames),
                                              'projections',
                                              'of',
                                              str(len(Projections)),
                                              str(WidthOfOutput),
                                              'px',
                                              '%03d.jpg' % c])))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    plt.show()
print('Done')



In [ ]:
# Let's write the output image with the chosen width (and scaled height)
# Scale the output figure size exactly: https://stackoverflow.com/a/13714720/323100
# Write canvas containts to image: https://stackoverflow.com/a/14913405/323100
for c, reconstruction in enumerate(Reconstructions[numpy.round(numpy.linspace(0, len(Reconstructions) - 1, NumberOfFrames)).astype(int)]):
    # Generate a figure with the chosen width and accordingly scaled height
    fig = plt.figure(figsize=(WidthOfOutput / plt.rcParams['figure.dpi'],
                              round(WidthOfOutput * (reconstruction.shape[0] / reconstruction.shape[1])) / plt.rcParams['figure.dpi']),
                     dpi=plt.rcParams['figure.dpi'])
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    # Equalize histogram for display reasons
    # Rotate reconstruction for display reasons
    ax.imshow(skimage.exposure.equalize_adapthist(dask.array.rot90(reconstruction)))
    ax.add_artist(ScaleBar(Pixelsize, 'um', fixed_value=5, fixed_units='mm'))
    s = '_'
    fig.canvas.print_jpg(os.path.join(BaseFolder, Sample, 'mov', 'mov_rec',
                                      s.join([Sample,
                                              str(NumberOfFrames),
                                              'projections',
                                              'of',
                                              str(len(Projections)),
                                              str(WidthOfOutput),
                                              'px',
                                              '%03d.jpg' % c])))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    plt.show()
print('Done')

In [ ]:
print('Now you can copy the files from %s to "TalkFolder/movies/scan/projection"' % os.path.join(BaseFolder, Sample, 'mov', 'mov_proj'))

In [ ]:
print('Now you can copy the files from %s to "TalkFolder/movies/scan/reconstruction"' % os.path.join(BaseFolder, Sample, 'mov', 'mov_rec'))

In [ ]:
# Show Sinogram
plt.imshow(Projections[:,555,:])

In [ ]:
from skimage.transform import radon, rescale

In [ ]:
skimage.transform.radon(Reconstructions[:,:,0])

In [ ]:
plt.imshow(skimage.transform.radon(Reconstructions[333][:,:,0]))

In [ ]:
skimage.transform.radon_transform?